https://github.com/udacity/self-driving-car-sim

# 整体

In [27]:
str1 = "D:\jupyter\Homework\homework4\data"
str2 = "D:\jupyter\Homework\homework4\data\IMG\center_2019_09_24_17_28_44_464.jpg"
print(str2[35:])
import pandas as pd
df = pd.read_csv(r"data\driving_log.csv")

a = df.iloc[:,0:3]
a[:5]

IMG\center_2019_09_24_17_28_44_464.jpg


,D:\jupyter\Homework\homework4\data\IMG\center_2019_09_24_17_28_44_464.jpg,D:\jupyter\Homework\homework4\data\IMG\left_2019_09_24_17_28_44_464.jpg,D:\jupyter\Homework\homework4\data\IMG\right_2019_09_24_17_28_44_464.jpg
0,D:\jupyter\Homework\homework4\data\IMG\center_...,D:\jupyter\Homework\homework4\data\IMG\left_20...,D:\jupyter\Homework\homework4\data\IMG\right_2...
1,D:\jupyter\Homework\homework4\data\IMG\center_...,D:\jupyter\Homework\homework4\data\IMG\left_20...,D:\jupyter\Homework\homework4\data\IMG\right_2...
2,D:\jupyter\Homework\homework4\data\IMG\center_...,D:\jupyter\Homework\homework4\data\IMG\left_20...,D:\jupyter\Homework\homework4\data\IMG\right_2...
3,D:\jupyter\Homework\homework4\data\IMG\center_...,D:\jupyter\Homework\homework4\data\IMG\left_20...,D:\jupyter\Homework\homework4\data\IMG\right_2...
4,D:\jupyter\Homework\homework4\data\IMG\center_...,D:\jupyter\Homework\homework4\data\IMG\left_20...,D:\jupyter\Homework\homework4\data\IMG\right_2...


In [7]:
# -*- coding: utf-8 -*-

import numpy as np
from keras.optimizers import SGD, Adam
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten, PReLU
from keras.models import Sequential, Model
from keras import backend as K
from keras.regularizers import l2
import os.path
import csv
import cv2
import glob
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import json
from keras import callbacks
import math
import matplotlib.pyplot as plt

SEED = 13


def horizontal_flip(img, degree):
    '''
    按照50%的概率水平翻转图像
    img: 输入图像
    degree: 输入图像对于的转动角度
    '''
    choice = np.random.choice([0,1])
    if choice == 1:
        img,degree = cv2.flip(img,1), -degree
    return (img, degree)


def random_brightness(img, degree):
    '''
    随机调整输入图像的亮度， 调整强度于 0.1(变黑)和1(无变化)之间
    img: 输入图像
    degree: 输入图像对于的转动角度
    '''
    hsv = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    alpha = np.random.uniform(low=0.1,high=1.0,size=None)
    v = hsv[:,:,2]
    v = v * alpha
    hsv[:,:,2] = v.astype('uint8')
    rgb = cv2.cvtColor(hsv.astype('uint8'),cv2.COLOR_HSV2RGB)
    return (img, degree)


def left_right_random_swap(img_address, degree, degree_corr=1.0 / 4):
    '''
    随机从左， 中， 右图像中选择一张图像， 并相应调整转动的角度
    img_address: 中间图像的的文件路径
    degree: 中间图像对于的方向盘转动角度
    degree_corr: 方向盘转动角度调整的值
    '''
    swap = np.random.choice(['l','r','c'])
    if swap == 'l':
        img_address = img_address.replace('center','left')
        correct_label = np.arctan(math.tan(degree) + degree_corr)
        return (img_address, correct_label) 
    elif swap == 'r':
        img_address = img_address.replace('center','right')
        correct_label = np.arctan(math.tan(degree) - degree_corr)
        return (img_address, correct_label) 
    else:
        return (img_address, degree)


def discard_zero_steering(degrees, rate):
    '''
    从角度为零的index中随机选择部分index返回
    degrees: 输入的角度值
    rate: 丢弃率， 如果rate=0.8， 意味着80%的index会被返回
    '''
    steering_zero_idx = np.where(degrees == 0)
    steering_zero_idx = steering_zero_idx[0]
    sizedel = int(len(steering_zero_idx)*rate)
    return np.random.choice(steering_zero_idx,size=sizedel,replace=False)

def get_model(shape):
    '''
    预测方向盘角度: 以图像为输入, 预测方向盘的转动角度
    shape: 输入图像的尺寸, 例如(128, 128, 3)
    '''

    model = Sequential()
    
    model.add(Conv2D(8, (5, 5), strides=(1, 1), padding="valid", activation='relu', input_shape=shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(8, (5, 5), strides=(1, 1), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(16, (4, 4), strides=(1, 1), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(16, (5, 5), strides=(1, 1), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))

    #sgd = SGD(lr=0.01)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


def image_transformation(img_address, degree, data_dir):
    img_address, degree = left_right_random_swap(img_address, degree)
    img = cv2.imread(data_dir + img_address)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img, degree = random_brightness(img, degree)
    img, degree = horizontal_flip(img, degree)

    return (img, degree)


def batch_generator(x, y, batch_size, shape, training=True, data_dir='data/', monitor=True, yieldXY=True, discard_rate=0.95):
    """
    产生批处理的数据的generator
    x: 文件路径list
    y: 方向盘的角度
    training: 值为True时产生训练数据
              值为False时产生validation数据
    batch_size: 批处理大小
    shape: 输入图像的尺寸(高, 宽, 通道)
    data_dir: 数据目录, 包含一个IMG文件夹
    monitor: 保存一个batch的样本为 'X_batch_sample.npy‘ 和'y_bag.npy’
    yieldXY: 为True时, 返回(X, Y)
             为False时, 只返回 X only
    discard_rate: 随机丢弃角度为零的训练数据的概率
    """
    
    if training:
        y_bag = []
        x, y = shuffle(x, y)
        rand_zero_idx = discard_zero_steering(y, rate=discard_rate)
        new_x = np.delete(x, rand_zero_idx, axis=0)
        new_y = np.delete(y, rand_zero_idx, axis=0)
    else:
        new_x = x
        new_y = y
    
    offset = 0
    while True: 
        X = np.empty((batch_size, *shape))
        Y = np.empty((batch_size, 1))

        for example in range(batch_size):
            img_address, img_steering = new_x[example + offset], new_y[example + offset]
            img_address = img_address[35:]
            if training:
                img, img_steering = image_transformation(img_address, img_steering, data_dir)
            else:
                img = cv2.imread(data_dir + img_address)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            X[example,:,:,:] = cv2.resize(img[80:140, 0:320], (shape[0], shape[1]) ) / 255 - 0.5
            
            Y[example] = img_steering
            if training:
                y_bag.append(img_steering)
            
            '''
             到达原来数据的结尾, 从头开始
            '''
            if (example + 1) + offset > len(new_y) - 1:
                x, y = shuffle(x, y)
                rand_zero_idx = discard_zero_steering(y, rate=discard_rate)
                new_x = x
                new_y = y
                new_x = np.delete(new_x, rand_zero_idx, axis=0)
                new_y = np.delete(new_y, rand_zero_idx, axis=0)
                offset = 0
        if yieldXY:
            yield (X, Y)
        else:
            yield X

        offset = offset + batch_size
        if training:
            np.save('y_bag.npy', np.array(y_bag) )
            np.save('Xbatch_sample.npy', X ) 


if __name__ == '__main__':

    data_path = 'data/'
    with open(data_path + 'driving_log.csv', 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        log = []
        for row in file_reader:
            log.append(row)

    log = np.array( log )
    # 去掉文件第一行
    #log = log[1:,:] 
    
    # 判断图像文件数量是否等于csv日志文件中记录的数量
    ls_imgs = glob.glob(data_path+ 'IMG/*.jpg')
    assert len(ls_imgs) == len(log)*3, 'number of images does not match'

    # 使用20%的数据作为测试数据
    validation_ratio = 0.2
    shape = (128, 128, 3)
    batch_size =128
    nb_epoch = 20

    x_ = log[:, 0] 
    y_ = log[:, 3].astype(float)
    x_, y_ = shuffle(x_, y_)
    X_train, X_val, y_train, y_val = train_test_split(x_, y_, test_size=validation_ratio, random_state=SEED)

    print('batch size: {}'.format(batch_size))
    print('Train set size: {} | Validation set size: {}'.format(len(X_train), len(X_val)))
        
    samples_per_epoch = 2000 
    # 使得validation数据量大小为batch_size的整数陪
    nb_val_samples = len(y_val) - len(y_val)%batch_size
    model = get_model(shape)
    print(model.summary())

    save_best = callbacks.ModelCheckpoint('15_best.h5',monitor='val_loss',verbose=1,save_best_only=True,mode='min')
    early_stop = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=15,verbose=0,mode='auto')
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0.000001)
    callbacks_list = [early_stop,save_best,rl]

    history = model.fit_generator(batch_generator(X_train, y_train, batch_size, shape, training=True),
                                  steps_per_epoch = samples_per_epoch,
                                  validation_steps = nb_val_samples // batch_size,
                                  validation_data = batch_generator(X_val, y_val, batch_size, shape, 
                                                                  training=False, monitor=False),
                                  epochs=nb_epoch, verbose=1, callbacks=callbacks_list)

    '''
    with open('./trainHistoryDict.p', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()'''
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # 保存模型
    '''
    with open('model.json', 'w') as f:
            f.write( model.to_json() )
    model.save('model.h5')
    '''
    print('Done!')

batch size: 128
Train set size: 4036 | Validation set size: 1010
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 8)       608       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 58, 8)         1608      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 16)        2064      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 16)        0         
_______________________________________________________

d:\programdata\anaconda3\envs\learn\lib\site-packages\keras\callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/20
2000/2000 [==============================] - 1391s 696ms/step - loss: 0.0511 - val_loss: 0.0534

Epoch 00001: val_loss improved from inf to 0.05341, saving model to 15_best.h5
Epoch 2/20
2000/2000 [==============================] - 1345s 672ms/step - loss: 0.0158 - val_loss: 0.0824

Epoch 00002: val_loss did not improve from 0.05341
Epoch 3/20
 469/2000 [======>.......................] - ETA: 17:34 - loss: 0.0117

KeyboardInterrupt: 

# 测试

In [11]:
from keras.models import load_model
import argparse
import base64
import json

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
import cv2
# Fix error with Keras and TensorFlow
#import tensorflow as tf

#tf.python.control_flow_ops = tf

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


# initialize parameter to record steering angle
# idx = 0
# steering_pred = []

@sio.on('telemetry')
def telemetry(sid, data):
    # global idx
    # global steering_pred
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_array = np.asarray(image)
    # convert from BGR to RGB
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    # save frames locally
    # fname = 'test_img/fname' + str(idx) +'.jpg'
    # cv2.imwrite(fname, image_array)

    # resize image to match training data image size
    image_array = image_array[80:140, 0:320]
    image_array = cv2.resize(image_array, (128, 128)) / 255. - 0.5
    transformed_image_array = image_array[None, :, :, :]

    #
    # print(transformed_image_array.shape)
    # This model currently assumes that the features of the model are just the images. Feel free to change this.

    steering_angle = float(model.predict(transformed_image_array, batch_size=1))
    # update steering angle record
    # steering_pred.append(steering_angle)
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = 2

    # idx = idx + 1
    # np.save('test_steering.npy', np.array(steering_pred))
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
    '''
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument('model', type=str,
                        help='Path to model definition json. Model weights should be on the same path.')
    args = parser.parse_args()
    with open(args.model, 'r') as jfile:
        # NOTE: if you saved the file by calling json.dump(model.to_json(), ...)
        # then you will have to call:
        #
        #   model = model_from_json(json.loads(jfile.read()))\
        #
        # instead.
        model = model_from_json(jfile.read())

    model.compile("adam", "mse")
    weights_file = args.model.replace('json', 'h5')
    model.load_weights(weights_file)
    '''
    model = load_model('15_best.h5')
    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(3372) wsgi starting up on http://0.0.0.0:4567
(3372) accepted ('127.0.0.1', 6627)


connect  1f68a7c5f5c9409383e8887d23888607
-0.2420129030942917 2
-0.2492130547761917 2
-0.2492130547761917 2
-0.24168480932712555 2
-0.22442714869976044 2
-0.23828978836536407 2
-0.23828978836536407 2
-0.20036503672599792 2
-0.21430478990077972 2
-0.22798986732959747 2
-0.20941634476184845 2
-0.20941634476184845 2
-0.208058163523674 2
-0.2566318213939667 2
-0.27379855513572693 2
-0.27379855513572693 2
-0.18487705290317535 2
-0.18487705290317535 2
-0.0067690592259168625 2
0.017375105991959572 2
-0.049481697380542755 2
-0.049481697380542755 2
0.027008065953850746 2
-0.044423118233680725 2
0.06433530896902084 2
0.06433530896902084 2
0.00964127667248249 2
-0.03775767236948013 2
-0.03775767236948013 2
-0.1166955828666687 2
-0.1166955828666687 2
-0.07717935740947723 2
-0.02798309735953808 2
-0.06825665384531021 2
-0.11957891285419464 2
-0.11957891285419464 2
-0.09135768562555313 2
-0.0889163464307785 2
-0.09456774592399597 2
-0.09456774592399597 2
-0.1803170144557953 2
0.04249066114425659 2
-

0.15133771300315857 2
-0.1302710473537445 2
-0.01927625946700573 2
-0.01927625946700573 2
-0.2659112215042114 2
-0.2749423682689667 2
-0.2749423682689667 2
-0.1807670146226883 2
-0.1807670146226883 2
-0.06774230301380157 2
-0.06774230301380157 2
0.08694180101156235 2
0.057738542556762695 2
0.057738542556762695 2
-0.11849372088909149 2
-0.13279855251312256 2
0.06936777383089066 2
-0.17471671104431152 2
-0.22737713158130646 2
0.005536174401640892 2
-0.28635066747665405 2
-0.28635066747665405 2
-0.06437866389751434 2
0.07718122750520706 2
-0.1890915185213089 2
-0.24729324877262115 2
-0.24729324877262115 2
-0.06986918300390244 2
-0.1470719426870346 2
0.02629646472632885 2
0.1456451117992401 2
0.1456451117992401 2
-0.08489479869604111 2
-0.2813517153263092 2
-0.2813517153263092 2
-0.2597266137599945 2
0.24295030534267426 2
0.016442162916064262 2
0.1451767534017563 2
0.17588815093040466 2
0.17588815093040466 2
0.27205103635787964 2
0.0998808965086937 2
0.10381978750228882 2
0.103819787502288

0.1839047223329544 2
0.1839047223329544 2
0.005624027922749519 2
0.005624027922749519 2
-0.07521744817495346 2
-0.020617371425032616 2
-0.08040818572044373 2
-0.08040818572044373 2
0.2242502123117447 2
-0.1285686045885086 2
0.03799387812614441 2
-0.1389174908399582 2
-0.1747288554906845 2
-0.0038955938071012497 2
-0.0038955938071012497 2
-0.25109192728996277 2
-0.057615041732788086 2
-0.24576686322689056 2
0.06330835819244385 2
-0.07381835579872131 2
-0.24271635711193085 2
-0.24271635711193085 2
-0.12416072189807892 2
-0.12416072189807892 2
-0.1913636177778244 2
-0.1913636177778244 2
-0.22685958445072174 2
-0.23804698884487152 2
-0.14592725038528442 2
0.01862477697432041 2
0.01862477697432041 2
-0.10162851214408875 2
-0.10162851214408875 2
-0.3344957232475281 2
-0.3344957232475281 2
0.010768556967377663 2
-0.14373333752155304 2
-0.26713114976882935 2
-0.26713114976882935 2
-0.09927186369895935 2
-0.15956388413906097 2
-0.15956388413906097 2
0.08043552935123444 2
-0.31375908851623535 2


0.07821186631917953 2
0.13332702219486237 2
0.01897919550538063 2
-0.010462740436196327 2
-0.010462740436196327 2
0.006971707567572594 2
0.010034125298261642 2
0.15826921164989471 2
0.09006156772375107 2
0.09006156772375107 2
0.1604030877351761 2
0.1604030877351761 2
-0.03643656522035599 2
0.07409092038869858 2
0.07409092038869858 2
0.07409092038869858 2
-0.11006656289100647 2
-0.013259200379252434 2
-0.013259200379252434 2
-0.05929020047187805 2
-0.05929020047187805 2
0.10727165639400482 2
-0.08360881358385086 2
-0.08360881358385086 2
0.18626712262630463 2
0.18626712262630463 2
0.055883005261421204 2
0.055883005261421204 2
0.0010923407971858978 2
-0.025820983573794365 2
-0.2891237437725067 2
-0.05105339735746384 2
-0.630153477191925 2
-0.630153477191925 2
-0.32519760727882385 2
-0.2653566300868988 2
-0.17079423367977142 2
-0.3135124146938324 2
-0.32467392086982727 2
-0.12839379906654358 2
-0.12839379906654358 2
-0.5622811317443848 2
-0.41622650623321533 2
-0.2924332022666931 2
-0.4385

...

0.14299941062927246 2
0.14299941062927246 2
0.1874612271785736 2
0.3003185987472534 2
0.0680132806301117 2
0.0680132806301117 2
0.35486918687820435 2
0.032326601445674896 2
-0.0031622927635908127 2
-0.06897473335266113 2
0.007216671481728554 2
0.007216671481728554 2
0.08131298422813416 2
0.21321997046470642 2
0.3326762914657593 2
0.3326762914657593 2
0.11593365669250488 2
0.2446882575750351 2
0.2446882575750351 2
0.005585787817835808 2
-0.04974418133497238 2
-0.04974418133497238 2
-0.05561275780200958 2
-0.05561275780200958 2
-0.29679927229881287 2
-0.29679927229881287 2
-0.4148373305797577 2
-0.2758990228176117 2
-0.4644644856452942 2
-0.33678996562957764 2
-0.3744235038757324 2
-0.3744235038757324 2
-0.2972941994667053 2
-0.4510544240474701 2
-0.3899836540222168 2
-0.3899836540222168 2
-0.06399588286876678 2
-0.06399588286876678 2
-0.17203088104724884 2
-0.3117036521434784 2
-0.2995949983596802 2
-0.3014042377471924 2
-0.13009138405323029 2
-0.13009138405323029 2
-0.11243008077144623

0.06987407058477402 2
0.06987407058477402 2
0.12277558445930481 2
0.12277558445930481 2
0.2321748286485672 2
0.017375411465764046 2
0.14972931146621704 2
-0.021490348502993584 2
0.03465311601758003 2
0.03465311601758003 2
0.15457414090633392 2
0.11752939224243164 2
0.11752939224243164 2
0.11689966917037964 2
0.016981245949864388 2
-0.07235454022884369 2
-0.07235454022884369 2
-0.024559950456023216 2
0.2148815542459488 2
-0.08360989391803741 2
0.10742989927530289 2
0.10742989927530289 2
0.27849701046943665 2
0.013643356040120125 2
-0.15924209356307983 2
0.11508530378341675 2
0.11508530378341675 2
0.21504710614681244 2
0.032016485929489136 2
0.032016485929489136 2
0.20908989012241364 2
0.19171670079231262 2
0.19171670079231262 2
0.05969274044036865 2
0.05969274044036865 2
-0.08125841617584229 2
-0.08125841617584229 2
-0.00039659813046455383 2
0.10791763663291931 2
0.050248149782419205 2
0.3012392222881317 2
0.40428799390792847 2
0.14157648384571075 2
0.14157648384571075 2
-0.213712006807

0.22561471164226532 2
0.22561471164226532 2
0.4745762050151825 2
0.13186781108379364 2
0.326431006193161 2
0.6466933488845825 2
0.7296069860458374 2
0.04998011887073517 2
0.04998011887073517 2
-0.14496520161628723 2
-0.023913605138659477 2
-0.023913605138659477 2
0.16757026314735413 2
0.16757026314735413 2
0.5378798246383667 2
0.3823465406894684 2
0.40992477536201477 2
0.40992477536201477 2
0.09108622372150421 2
0.2902291715145111 2
0.2902291715145111 2
0.76925128698349 2
0.6157910823822021 2
0.7438367009162903 2
0.7438367009162903 2
0.5985613465309143 2
0.6866440773010254 2
0.6866440773010254 2
0.6555566787719727 2
0.6416862607002258 2
0.6416862607002258 2
0.6416862607002258 2
0.45901790261268616 2
0.310639888048172 2
0.30556806921958923 2
0.25562626123428345 2
0.25562626123428345 2
0.25562626123428345 2
0.32665911316871643 2
0.30385202169418335 2
0.29433155059814453 2
0.29433155059814453 2
0.2943069636821747 2
0.2943069636821747 2
0.2667149007320404 2
0.2818778455257416 2
0.281877845

-0.0038782935589551926 2
-0.01626407913863659 2
-0.01626407913863659 2
0.023052820935845375 2
0.023052820935845375 2
0.03360367566347122 2
0.06396783888339996 2
0.0665540099143982 2
0.0665540099143982 2
0.051307208836078644 2
0.051307208836078644 2
0.07932529598474503 2
0.07932529598474503 2
0.05983638018369675 2
0.05983638018369675 2
0.09098371863365173 2
0.09052862226963043 2
0.09366949647665024 2
0.09805358201265335 2
0.09805358201265335 2
0.0980018898844719 2
0.0961688905954361 2
0.08995302021503448 2
0.08995302021503448 2
0.09438656270503998 2
0.09438656270503998 2
0.09832145273685455 2
0.11239517480134964 2
0.10681527853012085 2
0.10681527853012085 2
0.08892520517110825 2
0.07370810210704803 2
0.07370810210704803 2
0.07454211264848709 2
0.07454211264848709 2
0.07742931693792343 2
0.07613170146942139 2
0.06807002425193787 2
0.07541045546531677 2
0.07541045546531677 2
0.0624992698431015 2
0.0624992698431015 2
0.06949422508478165 2
0.09877578914165497 2
0.07725072652101517 2
0.07725

127.0.0.1 - - [25/Sep/2019 00:12:17] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 161.956890
wsgi exiting
(3372) wsgi exited, is_accepting=True
